# Astro Lab Using Hertzsprung-Russell Diagrams

Based on UNL's Flash-based [HR Diagram explorer](https://astro.unl.edu/naap/hr/animations/hr.html) from their [Astronomy page](https://astro.unl.edu).

This is a [Bokeh app](https://docs.bokeh.org/en/latest/) showing HR Diagrams of sets of stars (e.g., 10,000 brightest and 10,000 closest stars) from the Hipparcos catalog.

Although I probably can't help you debug your program I would love to hear if you use this animation (or derivative) in your astronomy lessons or observations.

Good luck,

Stephen Shadle 🌌

swshadle@gmail.com

At the terminal/console, run with<br>
`bokeh serve --show HR-HIP-Bokeh-App.ipynb`

In [1]:
import pandas as pd
import numpy as np
from bokeh.layouts import column, row
from bokeh.models import (Button, LinearColorMapper, LogColorMapper, ColumnDataSource,
                          Div, HoverTool, Label, SingleIntervalTicker, Slider, DataRange1d,
                          WheelZoomTool, RadioGroup, ColorBar, LabelSet, CheckboxGroup,
                          Select, LogTicker, FuncTickFormatter, Paragraph, CustomJS)
from bokeh.palettes import Blues256, Oranges256, Turbo256, Greys256
from bokeh.plotting import figure, curdoc, show

Enable MathJax equation numbering

https://github.com/ipython-contrib/jupyter_contrib_nbextensions/tree/master/src/jupyter_contrib_nbextensions/nbextensions/equation-numbering<br>
(Really, just enabling `\tag{#}` for manual equation numbering.)

In [2]:
# %%javascript
# MathJax.Hub.Config({
#     TeX: { equationNumbers: { autoNumber: "AMS" } }
# });
# MathJax.Hub.Queue(
#   ["resetEquationNumbers", MathJax.InputJax.TeX],
#   ["PreProcess", MathJax.Hub],
#   ["Reprocess", MathJax.Hub]
# );

In [3]:
# check that data files are in the data subdirectory under the current working directory
# import os
# arr = os.listdir('./data')

# print('should find "I_239_selection.tsv" & "named_stars.csv"')
# for f in arr:
#     print(f)

In [4]:
# imported the hipparcos catalog from http://balbuceosastropy.blogspot.com/2014/03/construction-of-hertzsprung-russell.html
# read in hipparcos data and selected named stars
filename = './data/I_239_selection.tsv'
df = pd.read_table(filename, skiprows=49, sep=';', header=None, index_col=0,
                   names = ['HIP',  # Hipparcos catalog number
                            'Vmag', # visual magnitude
                            'RA',   # right ascension
                            'Dec',  # declination
                            'Plx',  # parallax in milliarcseconds
                            'B-V',  # color in B-V
                            'SpType'], # spectral type
                   skipfooter=1,
                   engine='python')
filename = './data/named_stars.csv'
named_stars = pd.read_table(filename, skiprows=1, sep=',', header=None, index_col=1,
                   names = ['Name', 'HIP', 'NotUsed'], skipfooter=1, engine='python')
named_stars.drop(['NotUsed'], axis=1, inplace=True) # drop 3rd (NotUsed) column from named_stars

In [5]:
# change any field with spaces to NaN values so we can easily clean all stars with blank fields
df = df.applymap(lambda x: np.nan if isinstance(x, str)
                       and (not x or x.isspace()) else x)

# discard rows with any missing data
df = df.dropna()

# merge hipparcos (df) with named_stars--we can do this now that we've gotten rid of missing data in df
df = df.merge(named_stars, how='left', on='HIP')

# now that df has a few named stars, fill in the hipparcos catalog # for stars with no names
HIPidx = pd.Index.to_series(df.index)
names = HIPidx.map('HIP {:d}'.format)
values = {'Name': names}
df.fillna(value=values, inplace=True)

# numeric data is currently stored as strings. convert to floating point numbers.
df['Vmag'] = df['Vmag'].astype(np.float)
df['RA']   = df['RA'].astype(np.float)
df['Dec']  = df['Dec'].astype(np.float)
df['Plx']  = df['Plx'].astype(np.float)
df['B-V']  = df['B-V'].astype(np.float)

In [6]:
# produce a comma-separated list of named stars with commented names for use in the next step
# for n in named_stars.index:
#     print(f'{n}, # {named_stars["Name"][n]}')

In [7]:
# all named stars are on the following list. any stars can be added or removed from the list
# of displayed simply by commenting or uncommenting them.
select_HIPs_2b_labeled = [#0, # The sun
#                 664, # Belenos         
#                 677, # Alpheratz
#                 746, # Caph
#                 1067, # Algenib
#                 1547, # Citadelle         
#                 2081, # Ankaa
#                 2247, # Felixvarela       
#                 2920, # Fulu              
#                 3179, # Shedir
#                 3419, # Diphda
#                 3479, # Cocibolca         
#                 3527, # Musica            
#                 3821, # Achird            
                3829, # Van Maanen 2
#                 4422, # Castula           
#                 4427, # γ-Cassiopeiae
#                 4780, # Solaris           
#                 5054, # Nenque            
#                 5348, # Wurren            
#                 5447, # Mirach
#                 5529, # Emiw              
#                 5737, # Revati            
#                 6411, # Adhil             
#                 6686, # Ruchbah           
#                 7097, # Alpherg           
#                 7251, # Bosona            
#                 7513, # Titawin           
#                 7588, # Achernar
#                 7607, # Nembus            
#                 8198, # Torcular          
#                 8375, # Itonda            
#                 8645, # Baten Kaitos      
#                 8796, # Mothallah         
#                 8832, # Mesarthim         
#                 8886, # Segin             
#                 8903, # Sheratan          
#                 9487, # Alrescha          
#                 9640, # Almaak
#                 9884, # Hamal
#                 10813, # Lionrock          
#                 10826, # Mira
                11767, # Polaris
#                 12191, # Buna              
#                 12706, # Kaffaljidhma      
#                 12961, # Koeia             
#                 13061, # Lilii Borea       
#                 13192, # Nushagak          
#                 13209, # Bharani           
#                 13268, # Miram             
#                 13288, # Angetenar         
#                 13357, # Helvetios         
#                 13701, # Azha              
#                 13847, # Acamar
#                 13993, # Ayeyarwady        
#                 14135, # Menkar
#                 14322, # Ebla              
#                 14576, # Algol
#                 14668, # Misam             
#                 14838, # Botein            
#                 14879, # Dalim             
#                 15197, # Zibal             
#                 15578, # Intan             
#                 15863, # Mirphak
#                 16076, # Veritate          
#                 16084, # Poerava           
#                 16537, # Ran               
#                 17096, # Tupi              
#                 17448, # Atik              
#                 17489, # Celaeno           
#                 17499, # Electra           
#                 17531, # Taygeta           
#                 17573, # Maia              
#                 17579, # Asterope          
#                 17608, # Merope            
#                 17702, # Alcyone
#                 17847, # Atlas             
#                 17851, # Pleione
#                 18543, # Zaurak
#                 18614, # Menkib            
#                 19587, # Beid              
#                 19849, # Keid              
#                 20205, # Prima Hyadum      
#                 20455, # Secunda Hyadum    
#                 20535, # Beemim            
#                 20889, # Ain               
#                 20894, # Chamukuy          
#                 21109, # Hoggar            
#                 21393, # Theemin           
                21421, # Aldebaran
#                 21594, # Sceptrum          
#                 22449, # Tabit             
#                 22491, # Mouhoun           
#                 23015, # Hassaleh          
#                 23416, # Almaaz            
#                 23453, # Saclateni         
#                 23767, # Haedus            
#                 23875, # Cursa             
#                 24003, # Mago              
                24186, # Kapteyn's Star
                24436, # Rigel
                24608, # Capella
                25336, # Bellatrix
#                 25428, # Alnath
#                 25606, # Nihal
#                 25930, # Mintaka
#                 25985, # Arneb
#                 26207, # Meissa            
#                 26241, # Hatysa            
#                 26311, # Alnilam
#                 26380, # Bubup             
#                 26451, # Tianguan          
#                 26634, # Phact             
#                 26727, # Alnitak
                27366, # Saiph
#                 27628, # Wazn              
                27989, # Betelgeuse
#                 28360, # Menkalinan
#                 28380, # Mahasim           
#                 29034, # Elkurud           
#                 29550, # Amadioha          
#                 29655, # Propus            
#                 30089, # Red Rectangle
#                 30122, # Furud             
#                 30324, # Mirzam
#                 30343, # Tejat
                30438, # Canopus
#                 30860, # Lucilinburhuc     
#                 30905, # Lusitania         
#                 31681, # Alhena
#                 31895, # Nosaxa            
#                 32246, # Mebsuta           
                32349, # Sirius
#                 32362, # Alzirr            
#                 32916, # Nervia            
#                 33579, # Adhara
#                 33719, # Citala            
#                 33856, # Unurgunite        
#                 34045, # Muliphein         
#                 34088, # Mekbuda           
#                 34444, # Wezen (δ-Canis Majoris)
#                 35550, # Wasat             
#                 35904, # Aludra (η-Canis Majoris)
#                 36188, # Gomeisa           
                36208, # Luyten's Star
#                 36850, # Castor
#                 37265, # Jishui            
                37279, # Procyon
#                 37284, # Ceibo
                37826, # Pollux
#                 38041, # Tapecue           
#                 38170, # Azmidi            
#                 39429, # Naos (ζ-Puppis)
#                 39757, # Tureis            
#                 39953, # γ²-Vel
#                 40167, # Tegmine           
#                 40526, # Tarf              
#                 40687, # Nasti             
#                 40881, # Piautos           
#                 41037, # Avior
#                 41075, # Alsciaukat        
#                 41704, # Muscida           
#                 42402, # Minchir           
#                 42446, # Gakyid            
#                 42556, # Meleph            
#                 42806, # Asellus Borealis  
#                 42911, # Asellus Australis 
#                 42913, # Alsephina (δ-Velorum)
#                 43109, # Ashlesha          
#                 43587, # Copernicus        
#                 43674, # Stribor           
#                 44066, # Acubens           
#                 44127, # Talitha           
#                 44471, # Alkaphrah         
#                 44816, # Suhail (λ-Vela)
#                 44946, # Nahn              
#                 45238, # Miaplacidus
#                 45556, # Aspidiske (ι-Carina)
#                 45941, # κ-Vel
#                 46390, # Alphard
#                 46471, # Intercrus         
#                 46750, # Alterf            
#                 47087, # Illyrian          
#                 47202, # Kalausi           
#                 47431, # Ukdah             
#                 47508, # Subra             
#                 48235, # Natasha           
#                 48356, # Zhang             
#                 48455, # Rasalas           
#                 48615, # Felis             
#                 48711, # Bibha             
                49669, # Regulus
#                 50335, # Adhafera          
#                 50372, # Tania Borealis    
#                 50583, # Algieba
#                 50801, # Tania Australis   
#                 52521, # Macondo           
#                 53229, # Praecipua         
#                 53721, # Chalawan          
#                 53740, # Alkes             
#                 53910, # Merak
#                 54061, # Dubhe
#                 54158, # Dingolay          
#                 54872, # Zosma (δ-Leo)
#                 54879, # Chertan           
#                 55174, # Hunahpu           
#                 55203, # Alula Australis   
#                 55219, # Alula Borealis    
#                 55664, # Shama             
#                 56211, # Giausar           
#                 56508, # Formosa           
#                 56572, # Sagarmatha        
#                 57291, # Uklun             
#                 57370, # Flegetonte        
#                 57399, # Taiyangshou       
#                 57632, # Denebola
#                 57757, # Zavijava          
#                 57820, # Aniara            
#                 57939, # Groombridge 1830
#                 58001, # Phad
#                 58952, # Tonatiuh          
#                 59196, # δ-Centauri
#                 59199, # Alchiba           
#                 59747, # Imai              
#                 59774, # Megrez
#                 59803, # Gienah (γ-Corvi)
#                 60129, # Zaniah            
#                 60260, # Ginan             
#                 60644, # Tupa              
#                 60718, # Acrux (α-Crucis)
#                 60936, # 3C 273
#                 60965, # Algorab           
#                 61084, # γ-Crucis
#                 61177, # Funi              
#                 61317, # Chara             
#                 61359, # Kraz              
#                 61932, # γ-Centauri
#                 61941, # Porrima           
#                 62223, # La Superba        
#                 62423, # Tianyi            
#                 62434, # Mimosa
#                 62956, # Alioth
#                 63076, # Taiyi             
#                 63090, # Minelauva         
#                 63125, # Cor Caroli
#                 63608, # Vindemiatrix
#                 64241, # Diadem            
#                 65378, # Mizar
                65474, # Spica
#                 65477, # Alcor
#                 66047, # Dofida            
#                 66192, # Liesma            
#                 66249, # Heze              
#                 66657, # ε-Centauri
#                 67301, # Alkaid
#                 67927, # Muphrid           
#                 68002, # ζ-Centauri
#                 68702, # Agena
#                 68756, # Thuban
#                 68933, # Menkent (θ-Centauri)
#                 69427, # Kang              
                69673, # Arcturus
#                 69701, # Syrma             
#                 69732, # Xuange            
#                 69974, # Khambalia         
#                 70755, # Elgafar           
                70890, # Proxima
#                 71075, # Seginus           
#                 71352, # η-Centauri
#                 71681, # α-Centauri
                71683, # Rigil Kent
#                 71860, # α-Lupi
#                 72105, # Izar
#                 72339, # Monch             
#                 72487, # Merga             
#                 72607, # Kocab
#                 72622, # Zubenelgenubi     
#                 72845, # Arcalis           
#                 73136, # Baekdu            
#                 73555, # Nekkar            
#                 73714, # Brachium          
#                 74785, # Zubeneschamali    
#                 74961, # Nikawiy           
#                 75097, # Pherkad           
#                 75411, # Alkalurops        
#                 75458, # Edasich           
#                 75695, # Nusakan           
#                 76267, # Alphekka
#                 76333, # Zubenelhakrabi    
#                 76351, # Karaka            
#                 77070, # Unukalhai
#                 77450, # Gudja             
#                 78104, # Iklil             
#                 78265, # Fang              
#                 78401, # Dschubba (δ-Scorpii)
#                 78820, # Acrab             
#                 79043, # Marsic            
#                 79219, # Kamuy             
#                 79374, # Jabbah            
#                 79431, # Sharjah           
#                 79593, # Yed Prior         
#                 79882, # Yed Posterior     
#                 80076, # Hunor             
#                 80112, # Alniyat           
#                 80331, # Athebyne          
#                 80463, # Cujam             
#                 80687, # Timir             
                80763, # Antares
#                 80816, # Kornephoros       
#                 80838, # Ogma              
#                 80883, # Marfik            
#                 81022, # Rosaliadecastro   
#                 81266, # Paikauhale        
#                 82273, # Atria
#                 82396, # Larawag           
#                 82514, # Xamidimura        
#                 82545, # Pipirima          
#                 82651, # Mahsati           
#                 83547, # Rapeto            
#                 83608, # Alrakis           
#                 83895, # Aldhibah          
#                 84012, # Sabik             
#                 84345, # Rasalgethi
#                 84379, # Sarin             
#                 84405, # Guniibuu          
#                 84787, # Inquill           
#                 85670, # Rastaban          
#                 85693, # Maasym            
#                 85696, # Lesath            
#                 85822, # Yildun            
#                 85927, # Shaula
#                 86032, # Rasalhague
#                 86228, # Sargas (θ-Scorpii)
#                 86614, # Dziban            
#                 86742, # Cebalrai          
#                 86782, # Alruba            
#                 86796, # Cervantes         
#                 87261, # Fuyue             
#                 87585, # Grumium           
#                 87833, # Etamin
                87937, # Barnard's Star
#                 88414, # Pincoya           
#                 88635, # Alnasl            
#                 89341, # Polis             
#                 89931, # Kaus Media        
#                 90004, # Alasia            
#                 90185, # Kaus Australis
#                 90344, # Fafnir            
#                 90496, # Kaus Borealis     
                91262, # Vega
#                 91852, # Xihe              
#                 92420, # Sheliak
#                 92761, # Ainalrami         
#                 92855, # Nunki
#                 92895, # Kaveh             
#                 92946, # Alya              
#                 93194, # Sulafat           
#                 93506, # Ascella           
#                 93747, # Okab              
#                 94114, # Meridiana         
#                 94141, # Albaldah          
#                 94376, # Altais            
#                 94481, # Aladfar           
#                 94645, # Gumala            
#                 95124, # Belel             
#                 95241, # Arkab Prior       
#                 95262, # Sika              
#                 95294, # Arkab Posterior   
#                 95347, # Rukbat            
#                 95771, # Anser             
#                 95947, # Albireo
#                 96078, # Uruk              
#                 96100, # Alsafi            
#                 96295, # Campbell's Star
#                 96757, # Sham              
#                 97165, # Fawaris           
#                 97278, # Tarazed
#                 97649, # Altair
#                 97938, # Libertas          
#                 98036, # Alshain
#                 98066, # Terebellum        
#                 98298, # Cyg X-1
#                 99711, # Phoenicia         
#                 99894, # Chechia           
#                 100064, # Algedi            
#                 100310, # Alshat            
#                 100345, # Dabih             
#                 100453, # Sadr              
#                 100751, # Peacock
#                 101421, # Aldulfin          
#                 101769, # Rotanev           
#                 101958, # Sualocin          
                102098, # Deneb
#                 102488, # Aljanah           
#                 102618, # Albali            
#                 104382, # Polaris Australis 
#                 104987, # Kitalpha          
#                 105199, # Alderamin
#                 106032, # Alfirk            
#                 106278, # Sadalsuud         
#                 106786, # Bunda             
#                 106824, # Samaya            
#                 106985, # Nashira           
#                 107136, # Azelfafage        
#                 107315, # Enif
#                 107556, # Deneb Algedi      
#                 108085, # Aldhanab          
#                 108917, # Kurhah            
#                 109074, # Sadalmelik
#                 109268, # Alnair
#                 109427, # Biham             
#                 110003, # Ancha             
#                 110395, # Sadachbia         
#                 110893, # Kruger 60
#                 111710, # Situla            
#                 112029, # Homam             
#                 112122, # Tiaki             
#                 112158, # Matar             
#                 112247, # Babcock's Star
#                 112748, # Sadalbari         
#                 113136, # Skat              
                113368, # Fomalhaut
#                 113881, # Scheat
#                 113889, # Fumalsamakah      
#                 113963, # Markab
#                 115250, # Salm              
#                 115623, # Alkarab           
#                 116727, # Errai
               ]

In [8]:
stars_2b_labeled = named_stars[named_stars.index.isin(select_HIPs_2b_labeled)]

In [9]:
# manually add one important named star that's not in the Hipparcos catalog: the sun
stars_2b_labeled.loc[0] = 'The Sun'

/Users/Stavi/.pyenv/versions/miniconda3-4.7.12/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [10]:
stars_2b_labeled

,Name
HIP,
3829,Van Maanen 2
11767,Polaris
21421,Aldebaran
24186,Kapteyn's Star
24436,Rigel
24608,Capella
25336,Bellatrix
27366,Saiph
27989,Betelgeuse


In [11]:
# how many named stars?
# print(len(df[df['Name'].str.contains('HIP')==False]), 'named stars') # 423 named stars

In [12]:
# discarding stars with negative parallax: do we lose any named stars?
print('Named stars with unusable data (nonpositive parallax):')
count = 0
for name in df.loc[(df['Name'].str.contains('HIP')==False) & (df['Plx'] <= 0.0)]['Name']:
    count += 1
    print(name)
if not count:
    print('None')

Named stars with unusable data (nonpositive parallax):
Campbell's Star


In [13]:
df = df[df.index!=62223] # dumping "outlier" La Superba for being too red and messing up my colorbar

In [14]:
# discard stars with negative parallax
df = df[df['Plx'] > 0.0]

Solving for absolute magnitude, $M_V$, from apparent magnitude, $m_v$, which we have in data column `'Vmag'`.<br>
We'll use the formula for the distance modulus, $m_v - M_V$:
$$m_v - M_V = 5\cdot\log{d} - 5\tag{1}$$<br>
$$m_v - 5\cdot\log{d} + 5 = M_V$$<br>
Note that $d = 1/\omega$, where $d$ is distance measured in parsecs and $\omega$ is parallax measured in arcseconds.<br>
Our data column, `Plx`, is parallax measured in milliarcseconds, so $\omega = $ `Plx` $/1000$ and $d = 1000/$`Plx`<br>
$$m_v - 5\cdot\log{\frac{1}{\omega}} + 5 = M_V$$<br>
$$m_v - 5\cdot\log{\frac{1000}{\mathtt{Plx}}} + 5 = M_V$$<br>
$$m_v - 5(\log{1000} - \log{\mathtt{Plx}}) + 5 = M_V$$<br>
$$m_v - 5(3 - \log{\mathtt{Plx}}) + 5 = M_V$$<br>
$$m_v - 15 + 5\cdot\log{\mathtt{Plx}} + 5 = M_V$$<br>
$$m_v + 5\cdot\log{\mathtt{Plx}} - 10 = M_V\tag{2}$$

In [15]:
# calculate absolute magnitude and add a new column ('M_V')
df['M_V'] = df['Vmag'] + 5*np.log10(df['Plx'])-10

Calculate distance in LY and add a new data column<br>
$d = 1/p$ when distance ($d$) is in parsecs and parallax ($p$) is in arcseconds, or $d = 1000/\mathtt{Plx}$<br>
Note that Hipparcos parallax data (column `'Plx'`) is in milliarcseconds (MAS), so we multiply
by 1000 to get arcseconds).<br>
Finally, there are 3.26156 light-years (LY) in one parsec, so the final formula is $LY = 3261.56/\mathtt{Plx}$

In [16]:
df['LY'] = 3261.56/df['Plx']

In [17]:
sun_ly = 0.000015639
sun_plx = 3261.56/sun_ly

Now that we have absolute magnitude (column `'M_V'`), we can solve for luminosity in terms of multiples of luminosity of the sun, or $\frac{L}{L_{sun}}$, using:
$$M_V = M_{Vsun} - 2.5 \cdot log\frac{L}{L_{sun}}\tag{3}$$<br>
$$2.5 \cdot log\frac{L}{L_{sun}} = M_{Vsun} - M_V$$<br>
$$log\frac{L}{L_{sun}} = \frac{M_{Vsun} - M_V}{2.5}$$<br>
$$\frac{L}{L_{sun}} = 10^{\frac{M_{Vsun} - M_V}{2.5}}\tag{4}$$<br>
http://hosting.astro.cornell.edu/academics/courses/astro201/mag_absolute.htm

In [18]:
# add a new column ('L_sun') for luminosity in terms of multiples of solar luminosity
df['L_sun'] = np.power(10,[(4.83-m)/2.5 for m in df['M_V']]) # M_V_☉ = 4.83

Solving for effective temperature in Kelvin, $T_K$, using:
$$T_K = \frac{5601 K}{(B-V+0.4)^{2/3}}\tag{5}$$<br>
Note that this formula breaks when $B-V = -0.4$<br>
http://astro.physics.uiowa.edu/ITU/labs/professional-labs/photometry-of-a-globular/part-2-finding-temperature.html

In [19]:
# print("We'll have to lose", len(df[df['B-V']<=-0.4]), "star(s) because of T_K formula (B-V can't be <= 0.4)")
# print('# of stars before:', len(df))

In [20]:
df = df[df['B-V']>-0.4]

In [21]:
# print('# of stars after:', len(df))

In [22]:
# add a new column ('T_K') for effective temperature in K from color index ('B-V')
df['T_K'] = [5601/np.power(c+0.4,2/3) for c in df['B-V']]

We want to show slider values with a marker on the HR plot. The sliders' temperature and luminosity can be converted to the HR plot's color index (x-axis) and abs. magnitude (y-axis) using equation (3) for `M_V` and solving equation (5) for `B-V`.<br>
$$M_V = M_{Vsun} - 2.5 \cdot log\frac{L}{L_{sun}}\tag{3}$$<br>
$$T_K = \frac{5601 K}{(B-V+0.4)^{2/3}}\tag{5}$$<br>
$$(B-V+0.4)^{2/3} = \frac{5601 K}{T_K}$$<br>
$$B-V+0.4 = \left(\frac{5601 K}{T_K}\right)^{3/2}$$<br>
$$B-V = \left( \frac{5601 K}{T_K} \right) ^{3/2} - 0.4\tag{6}$$

Now that we have $T_K$ and $\frac{L}{L_{sun}}$, we can solve for $\frac{R}{R_{sun}}$, using<br>
$$\frac{R}{R_{sun}} = \frac{\sqrt{L/L_{sun}}}{(T_K/T_{sun})^2}\tag{7}$$

In [23]:
# temperature in K, B-V color index, and abs. magnitude of the sun
T_sun = 5780
BV_sun = np.power(5601/T_sun, 1.5) - 0.4
M_sun = 4.83

In [24]:
# add a new column ('R_sun') for radius in units of solar radii from luminosity and effective temperature in K
df['R_sun']=np.around(np.sqrt(df['L_sun'])/(df['T_K']/T_sun)**2, decimals=2) # T_⊙ = 5780

In [30]:
# more data cleaning. which named stars have the largest calculated solar radii?
df.nlargest(400,'R_sun').loc[(df['Name'].str.contains('HIP')==False)] #Polis 748.67 Antares 326.10

,Vmag,RA,Dec,Plx,B-V,SpType,Name,M_V,LY,L_sun,T_K,R_sun
HIP,,,,,,,,,,,,
89341,3.84,273.440870,-21.058830,0.11,0.195,B2III:,Polis,-10.953037,29650.545455,2.056907e+06,7917.491313,764.34
80763,1.06,247.351948,-26.431946,5.40,1.865,M1Ib + B2.5V,Antares,-5.278031,603.992593,1.104619e+04,3247.531970,332.93


In [26]:
# what is Antares's radius?
float(df[df['Name']=='Antares']['R_sun'])

332.93

In [27]:
df[df['R_sun']==float(df[df['Name']=='Antares']['R_sun'])] # horrible way to find a star but a good way to limit a database to R_suns <= Antares's

,Vmag,RA,Dec,Plx,B-V,SpType,Name,M_V,LY,L_sun,T_K,R_sun
HIP,,,,,,,,,,,,
80763,1.06,247.351948,-26.431946,5.4,1.865,M1Ib + B2.5V,Antares,-5.278031,603.992593,11046.189271,3247.53197,332.93


In [31]:
df.loc[(df['Name'].str.contains('HIP')==False) & (df['R_sun']>323.86)]

,Vmag,RA,Dec,Plx,B-V,SpType,Name,M_V,LY,L_sun,T_K,R_sun
HIP,,,,,,,,,,,,
80763,1.06,247.351948,-26.431946,5.40,1.865,M1Ib + B2.5V,Antares,-5.278031,603.992593,1.104619e+04,3247.531970,332.93
89341,3.84,273.440870,-21.058830,0.11,0.195,B2III:,Polis,-10.953037,29650.545455,2.056907e+06,7917.491313,764.34


In [33]:
len(df) # length of dataframe before

110714

In [34]:
# just suspicious of calculated radii for the largest stars. i'm cutting out stars larger than Antares but you might choose to leave them in.
df = df[df['R_sun']<=float(df[df['Name']=='Antares']['R_sun'])]

In [35]:
len(df) # and after

110389

In [42]:
# add the sun's particulars to the dataframe
df.loc[0] = {'Vmag': -26.74,
             'RA': 0.0,
             'Dec': 0.0,
             'Plx': sun_plx,
             'B-V': BV_sun, # 0.656
             'SpType': 'G2V',
             'Name': 'The Sun',
             'M_V': M_sun, # 4.83
             'LY': sun_ly,
             'L_sun': 1.0,
             'T_K': float(T_sun),#5780.0,
             'R_sun': 1.0}

In [43]:
# scale marker sizes based on star sizes
sizemax = df['R_sun'].max()
sizemin = df['R_sun'].min()

df['markersizes'] = [50*(r-sizemin)/(sizemax-sizemin)+2 for r in df['R_sun']]

In [44]:
df_close = df.nsmallest(10000,'LY') # get 10k of the closest stars
df_bright = df.nsmallest(10000,'Vmag') # get 10k of the brightest stars
df_either = pd.merge(df_bright, df_close, how='outer') # stars that are either bright or close
df_both   = pd.merge(df_bright, df_close, how='inner') # stars that are both bright and close
df_named        = df[df['Name'].str.contains('HIP')==False] # get all stars with common names (without HIP #--hipparcos number designators)

df_bright_labels = pd.merge(df_bright, stars_2b_labeled, how='inner')[['B-V', 'M_V', 'Name']]
df_close_labels  = pd.merge(df_close,  stars_2b_labeled, how='inner')[['B-V', 'M_V', 'Name']]
df_either_labels = pd.merge(df_either, stars_2b_labeled, how='inner')[['B-V', 'M_V', 'Name']]
df_both_labels   = pd.merge(df_both,   stars_2b_labeled, how='inner')[['B-V', 'M_V', 'Name']]
df_named_labels  = pd.merge(df_named,  stars_2b_labeled, how='inner')[['B-V', 'M_V', 'Name']]

print(len(df_bright), 'bright stars')
print(len(df_close),  'close stars')
print(len(df_either), 'stars that are either bright or close')
print(len(df_both),   'stars that are both bright and close')
print(len(df_named),  'total named stars')

10000 bright stars
10000 close stars
17997 stars that are either bright or close
2003 stars that are both bright and close
421 total named stars


In [45]:
# make a label for the bottom-center of the star plot showing which set of star. useful when saving a view to disk.
label = Label(x=350, y=0, x_units='screen', y_units='screen', text_align='center',#'data'
              background_fill_color='black', background_fill_alpha=1, text='')

In [46]:
# callback for the radiobutton group
def zero():
#     print('button zero')
    near_plot.visible         = True
    bright_plot.visible       = False
    both_plot.visible         = False
    either_plot.visible       = False
    named_plot.visible        = False

    close_lum_labels.visible  = 1 in checkboxgroup.active
    bright_lum_labels.visible = False
    both_lum_labels.visible   = False
    either_lum_labels.visible = False
    named_lum_labels.visible  = False

    if 0 in checkboxgroup.active:
        close_labels.visible         = True
        close_labels_shadow.visible  = True
        bright_labels.visible        = False
        bright_labels_shadow.visible = False
        both_labels.visible          = False
        both_labels_shadow.visible   = False
        either_labels.visible        = False
        either_labels_shadow.visible = False
        named_labels.visible         = False
        named_labels_shadow.visible  = False
        
    plot.x_range = DataRange1d(only_visible=True, range_padding=0.1, range_padding_units='percent') # 'absolute'
    plot.y_range = DataRange1d(only_visible=True, range_padding=0.1, range_padding_units='percent', flipped=True)
    
    return f'{len(df_close)} nearest stars '

def one():
#     print('button one')
    near_plot.visible         = False
    bright_plot.visible       = True
    both_plot.visible         = False
    either_plot.visible       = False
    named_plot.visible        = False
    
    close_lum_labels.visible  = False
    bright_lum_labels.visible = 1 in checkboxgroup.active
    both_lum_labels.visible   = False
    either_lum_labels.visible = False
    named_lum_labels.visible  = False

    if 0 in checkboxgroup.active:
        close_labels.visible         = False
        close_labels_shadow.visible  = False
        bright_labels.visible        = True
        bright_labels_shadow.visible = True
        both_labels.visible          = False
        both_labels_shadow.visible   = False
        either_labels.visible        = False
        either_labels_shadow.visible = False
        named_labels.visible         = False
        named_labels_shadow.visible  = False

    plot.x_range = DataRange1d(only_visible=True, range_padding=0.1, range_padding_units='percent') # 'absolute'
    plot.y_range = DataRange1d(only_visible=True, range_padding=1.0, range_padding_units='absolute', flipped=True)

    return f'{len(df_bright)} brightest stars '

def two():
#     print('button two')
    near_plot.visible         = False
    bright_plot.visible       = False
    both_plot.visible         = True
    either_plot.visible       = False
    named_plot.visible        = False

    close_lum_labels.visible  = False
    bright_lum_labels.visible = False
    both_lum_labels.visible   = 1 in checkboxgroup.active
    either_lum_labels.visible = False
    named_lum_labels.visible  = False

    if 0 in checkboxgroup.active:
        close_labels.visible         = False
        close_labels_shadow.visible  = False
        bright_labels.visible        = False
        bright_labels_shadow.visible = False
        both_labels.visible          = True
        both_labels_shadow.visible   = True
        either_labels.visible        = False
        either_labels_shadow.visible = False
        named_labels.visible         = False
        named_labels_shadow.visible  = False

    plot.x_range = DataRange1d(only_visible=True, range_padding=0.1, range_padding_units='percent') # 'absolute'
    plot.y_range = DataRange1d(only_visible=True, range_padding=1.0, range_padding_units='absolute', flipped=True)

    return f'{len(df_both)} near and bright stars '

def three():
#     print('button three')
    near_plot.visible         = False
    bright_plot.visible       = False
    both_plot.visible         = False
    either_plot.visible       = True
    named_plot.visible        = False

    close_lum_labels.visible  = False
    bright_lum_labels.visible = False
    both_lum_labels.visible   = False
    either_lum_labels.visible = 1 in checkboxgroup.active
    named_lum_labels.visible  = False

    if 0 in checkboxgroup.active:
        close_labels.visible         = False
        close_labels_shadow.visible  = False
        bright_labels.visible        = False
        bright_labels_shadow.visible = False
        both_labels.visible          = False
        both_labels_shadow.visible   = False
        either_labels.visible        = True
        either_labels_shadow.visible = True
        named_labels.visible         = False
        named_labels_shadow.visible  = False

    plot.x_range = DataRange1d(only_visible=True, range_padding=0.1, range_padding_units='percent') # 'absolute'
    plot.y_range = DataRange1d(only_visible=True, range_padding=1.0, range_padding_units='absolute', flipped=True)

    return f'{len(df_either)} either near or bright stars '

def four():
#     print('button four')
    near_plot.visible         = False
    bright_plot.visible       = False
    both_plot.visible         = False
    either_plot.visible       = False
    named_plot.visible        = True

    close_lum_labels.visible  = False
    bright_lum_labels.visible = False
    both_lum_labels.visible   = False
    either_lum_labels.visible = False
    named_lum_labels.visible  = 1 in checkboxgroup.active

    if 0 in checkboxgroup.active:
        close_labels.visible         = False
        close_labels_shadow.visible  = False
        bright_labels.visible        = False
        bright_labels_shadow.visible = False
        both_labels.visible          = False
        both_labels_shadow.visible   = False
        either_labels.visible        = False
        either_labels_shadow.visible = False
        named_labels.visible         = True
        named_labels_shadow.visible  = True

    plot.x_range = DataRange1d(only_visible=True, range_padding=0.1, range_padding_units='percent') # 'absolute'
    plot.y_range = DataRange1d(only_visible=True, range_padding=1.0, range_padding_units='absolute', flipped=True)

    return f'{len(df_named)} stars with common names '

def radiogroup_callback(new):
    switcher = {
        0: zero,
        1: one,
        2: two,
        3: three,
        4: four
    }
    func = switcher.get(new, lambda: 'Invalid selection') # get the function from switcher dictionary
    label.text = func() # execute the function

def checkboxgroup_callback(new):
    if 0 in new:
#         print('item 1 is checked')
        if radiogroup.active==0:
            close_labels.visible         = True
            close_labels_shadow.visible  = True
        if radiogroup.active==1:
            bright_labels.visible        = True
            bright_labels_shadow.visible = True
        if radiogroup.active==2:
            both_labels.visible          = True
            both_labels_shadow.visible   = True
        if radiogroup.active==3:
            either_labels.visible        = True
            either_labels_shadow.visible = True
        if radiogroup.active==4:
            named_labels.visible         = True
            named_labels_shadow.visible  = True
    else:
        close_labels.visible         = False
        close_labels_shadow.visible  = False
        bright_labels.visible        = False
        bright_labels_shadow.visible = False
        both_labels.visible          = False
        both_labels_shadow.visible   = False
        either_labels.visible        = False
        either_labels_shadow.visible = False
        named_labels.visible         = False
        named_labels_shadow.visible  = False
            
    if 1 in new:
#         print('item 2 is checked')
        isoradial_lines.visible = True

        if radiogroup.active==0:
            close_lum_labels.visible  = True
        if radiogroup.active==1:
            bright_lum_labels.visible = True
        if radiogroup.active==2:
            both_lum_labels.visible   = True
        if radiogroup.active==3:
            either_lum_labels.visible = True
        if radiogroup.active==4:
            named_lum_labels.visible  = True
    else:
        isoradial_lines.visible = False

        close_lum_labels.visible  = False
        bright_lum_labels.visible = False
        both_lum_labels.visible   = False
        either_lum_labels.visible = False
        named_lum_labels.visible  = False

In [47]:
# CheckboxGroup widget
checkboxgroup_labels = ['Show star names', 'Show isoradial lines']
checkboxgroup = CheckboxGroup(labels=checkboxgroup_labels, active=[0])
checkboxgroup.on_click(checkboxgroup_callback)

# RadioGroup widget
radiogroup_labels = ['Near stars', 'Bright stars', 'Both near and bright', 'Either near or bright', 'Named stars']
radiogroup = RadioGroup(labels=radiogroup_labels, active=0)
radiogroup.on_click(radiogroup_callback)

In [48]:
source_close  = ColumnDataSource(df_close) ; label.text = f'{len(df_close)} nearest stars '
source_close_labels = ColumnDataSource(df_close_labels)

source_bright = ColumnDataSource(df_bright)#; label.text = f'{len(df_bright)} brightest stars '
source_bright_labels = ColumnDataSource(df_bright_labels)

source_both   = ColumnDataSource(df_both)  #; label.text = f'{len(df_both)} near and bright stars '
source_both_labels = ColumnDataSource(df_both_labels)

source_either = ColumnDataSource(df_either)#; label.text = f'{len(df_either)} either bright or near stars '
source_either_labels = ColumnDataSource(df_either_labels)

source_named  = ColumnDataSource(df_named) #; label.text = f'{len(df_named)} stars with common names '
source_named_labels = ColumnDataSource(df_named_labels)

source = source_close

In [49]:
plot = figure(title=f'HR Diagram',
              tools='pan,zoom_in,zoom_out,save,reset',
              active_drag='pan',
              plot_width=650, plot_height=650)
plot.xaxis.ticker = plot.yaxis.ticker = SingleIntervalTicker(interval=1)
plot.xaxis.axis_label = 'B-V'
plot.yaxis.axis_label = 'Mᵥ'
plot.title.align = 'left'
plot.title.offset = .6
plot.title.text_font_size = '12pt'
plot.title.text_font = "times"
plot.title.text_font_style = "italic"

wheel_zoom = WheelZoomTool(zoom_on_axis=True)
plot.add_tools(wheel_zoom)
plot.toolbar.active_scroll = wheel_zoom

plot.background_fill_color = 'black'
plot.grid.grid_line_color = None
plot.toolbar.autohide = True
plot.toolbar.logo=None
plot.toolbar_location='above'#'below'

plot.x_range = DataRange1d(only_visible=True, range_padding=0.1, range_padding_units='percent') # 'absolute'
plot.y_range = DataRange1d(only_visible=True, range_padding=0.08, range_padding_units='percent', flipped=True)

plot.add_layout(label, place='left')

In [50]:
# make a color scheme for color index (B-V) colorbar
repeat_white = 10 # of colors, out of 256, reserved for white in the middle of the colorbar (makes white strip bigger)
skew_left_pct = 35 # percent of skew toward the left of the colorbar (bottom if vertical) (moves white strip left)
skew_left = int(256*(skew_left_pct/100)) # resulting # of colors to skew left out of 256

blu_or = [Blues256[b] for b in np.linspace(0, 255, (256-repeat_white)//2-skew_left, dtype=int, endpoint=True)] + ['white']*repeat_white \
     + [Oranges256[b] for b in np.linspace(255, 0, (256-repeat_white)//2+skew_left, dtype=int, endpoint=True)]

In [51]:
select_mapper = Select(title='Select color bar', width=160, value='Color index (B-V)',
                       options=['Color index (B-V)', 'Effective temperature', 'Luminosity'])

def update_mapper(attrname, old, new):
    if select_mapper.value == 'Color index (B-V)':
        colorbar_color.visible = True
        colorbar_temperature.visible = False
        colorbar_luminosity.visible = False
        near_plot.glyph.fill_color   = dict(field='B-V', transform=bv_color_mapper)
        bright_plot.glyph.fill_color = dict(field='B-V', transform=bv_color_mapper)
        both_plot.glyph.fill_color   = dict(field='B-V', transform=bv_color_mapper)
        either_plot.glyph.fill_color = dict(field='B-V', transform=bv_color_mapper)
        named_plot.glyph.fill_color  = dict(field='B-V', transform=bv_color_mapper)
    elif select_mapper.value == 'Effective temperature':
        colorbar_color.visible = False
        colorbar_temperature.visible = True
        colorbar_luminosity.visible = False
        near_plot.glyph.fill_color   = dict(field='T_K', transform=color_mapper_temperature)
        bright_plot.glyph.fill_color = dict(field='T_K', transform=color_mapper_temperature)
        both_plot.glyph.fill_color   = dict(field='T_K', transform=color_mapper_temperature)
        either_plot.glyph.fill_color = dict(field='T_K', transform=color_mapper_temperature)
        named_plot.glyph.fill_color  = dict(field='T_K', transform=color_mapper_temperature)
    else: # Luminosity
        colorbar_color.visible = False
        colorbar_temperature.visible = False
        colorbar_luminosity.visible = True
        near_plot.glyph.fill_color   = dict(field='L_sun', transform=color_mapper_luminosity)
        bright_plot.glyph.fill_color = dict(field='L_sun', transform=color_mapper_luminosity)
        both_plot.glyph.fill_color   = dict(field='L_sun', transform=color_mapper_luminosity)
        either_plot.glyph.fill_color = dict(field='L_sun', transform=color_mapper_luminosity)
        named_plot.glyph.fill_color  = dict(field='L_sun', transform=color_mapper_luminosity)

select_mapper.on_change('value', update_mapper)

To show isoradial lines, we combine equations (4), (5), and (7) to find absolute magnitude (`M_V`) on the y-axis in terms of color index (`B-V`) on the x-axis versus. For a set of lines where $\frac{R}{R_{sun}} = 0.01, 0.1, 1, 10, 100, 1000$, we'll set equation (7) equal to some factor, *F*, for orders of radial magnitude. For convenience, we'll make F an exponent because we'll end up taking the log.<br><br>
$$\frac{R}{R_{sun}} = 10^F = \frac{\sqrt{L/L_{sun}}}{(T_K/T_{sun})^2}$$<br>
$$10^F = \frac{\sqrt{L/L_{sun}}}{(T_K/T_{sun})^2}$$<br>
Subbing in equation (4), $T_K = \frac{5601 K}{(B-V+0.4)^{2/3}}$ and equation (5), $\frac{L}{L_{sun}} = 10^{\frac{M_{Vsun} - M_V}{2.5}}$, we get<br><br>
$$10^F = \frac{\sqrt{10^{\frac{M_{Vsun} - M_V}{2.5}}}}{\left(\frac{5601 K}{(B-V+0.4)^{2/3}}\big/T_{sun}\right)^2} =
\frac{\sqrt{10^{\frac{M_{Vsun} - M_V}{2.5}}}}{\left(\frac{5601 K/T_{sun}}{(B-V+0.4)^{2/3}}\right)^2}$$<br>
$$10^F\cdot\left(\frac{5601 K\big/T_{sun}}{(B-V+0.4)^{2/3}}\right)^2 = \sqrt{10^{\frac{M_{Vsun} - M_V}{2.5}}}$$<br>
Squaring both sides, we get $$10^{2F}\cdot\left(\frac{5601 K\big/T_{sun}}{(B-V+0.4)^{2/3}}\right)^4 = 10^{\frac{M_{Vsun} - M_V}{2.5}}$$<br>
Taking the $\log_{10}$ of both sides, we get $$\log{\bigg(10^{2F}\cdot\Big(\frac{5601 K\big/T_{sun}}{(B-V+0.4)^{2/3}}\Big)^4\bigg)} = \log{10^{\frac{M_{Vsun} - M_V}{2.5}}}$$<br>
$$2F+4\log{\frac{5601 K\big/T_{sun}}{(B-V+0.4)^{2/3}}} = \frac{M_{Vsun} - M_V}{2.5}$$<br>
$$5F+10\log{\frac{5601 K\big/T_{sun}}{(B-V+0.4)^{2/3}}} = M_{Vsun} - M_V$$<br>
$$M_V = M_{Vsun} - 5F-10\log{\frac{5601 K\big/T_{sun}}{(B-V+0.4)^{2/3}}}, F = \big\{n \in \mathbb{Z} \bigm\lvert -2\le n\le3\big\}\tag{8}$$<br>

In [52]:
# make dotted lines showing equal radius
F = [-2, -1, 0, 1, 2, 3]
x = [x for x in np.arange(df['B-V'].min(), df['B-V'].max(), 0.1)]
xs = [x, x, x, x, x, x]
ys = [[M_sun - 5*f - 10*np.log10((5601/T_sun)/np.power(c+0.4,2/3)) for c in x] for f in F]

rad_source = ColumnDataSource(data=dict(xs=xs, ys=ys))

isoradial_lines = plot.multi_line(
    xs='xs',
    ys='ys',
    line_alpha=0.9,
    line_cap='butt',
    line_color='white',
    line_dash='dashed',
    line_dash_offset=0,
    line_join='bevel',
    line_width=1,
    visible=False,
    source=rad_source)

In [54]:
# starting and ending slider values
start_lum, end_lum  = np.log10(0.00001), np.log10(10000000)
start_bv, end_bv = df['B-V'].min(), df_either['B-V'].max() # df_either['B-V'].min()

# ColorBar from B-V (color index)
bv_color_mapper = LinearColorMapper(palette=blu_or, low=start_bv, high=end_bv)#low=df['B-V'].min(), high=df['B-V'].max()) #Turbo256

# Categorical colors from list of unique eff. temperatures, rounded to nearest 1000
color_mapper_temperature = LogColorMapper(palette=Turbo256, low=df_close['T_K'].min(),   high=df_close['T_K'].max())
color_mapper_luminosity  = LogColorMapper(palette=Greys256, low=df_close['L_sun'].min(), high=df_close['L_sun'].max())

In [55]:
line_color='#7c7e71'#None

def scatter_factory(source, visible, name):
    return plot.scatter(x='B-V', y='M_V', size='markersizes', line_color=line_color,
                        fill_color={'field': 'B-V', 'transform': bv_color_mapper}, # Color index from B-V
                        line_width=0.25, line_alpha=0.75, fill_alpha=1,
                        source=source, visible=visible, name=name)

near_plot   = scatter_factory(source_close,  True,  'stars') # the name 'stars' lets the hovertips activate for stars only and ignore other glyphs on the plot, such as the slider indicator and solar radius labels
bright_plot = scatter_factory(source_bright, False, 'stars')
both_plot   = scatter_factory(source_both,   False, 'stars')
either_plot = scatter_factory(source_either, False, 'stars')
named_plot  = scatter_factory(source_named,  False, 'stars')

# adding orange-red asterisk to indicate slider values after the star plots so it shows up on top
# (under name labels is ok)
xy_source = ColumnDataSource(data=dict(x=[BV_sun], y=[M_sun])) # marker starts at the sun's x- and y-coordinates
render_marker = plot.scatter(x='x', y='y', size=15, marker='asterisk', line_color='orangered',#'azure','black'
                             source=xy_source)

# declaring range renderers allows the plot to ignore solar radius lines and labels from affecting the automatic range limits
plot.x_range.renderers = [near_plot, bright_plot, both_plot, either_plot, named_plot, render_marker]
plot.y_range.renderers = [near_plot, bright_plot, both_plot, either_plot, named_plot, render_marker]

In [56]:
def label_locations(lum, lt, rt, bm): # note luminosity (lum) is a power of 10
#     try for label on right within margins
    mag = M_sun - 5*lum - 10*np.log10((5601/T_sun)/np.power(rt+0.4,2/3))
    if mag>bm: # go with label on left instead
        return (lt, M_sun - 5*lum - 10*np.log10((5601/T_sun)/np.power(lt+0.4,2/3)))
    else:
        return (rt, mag)

def lum_label_factory(dataframe):
    left_margin   = dataframe['B-V'].min()
    right_margin  = dataframe['B-V'].max()
    bottom_margin = dataframe['M_V'].max()

    x = np.zeros(6)
    y = np.zeros(6)
    for i, lum in enumerate([-2, -1, 0, 1, 2, 3]):
        x[i], y[i] = label_locations(lum, left_margin, right_margin, bottom_margin)

    label_source = ColumnDataSource(data=dict(x=x, y=y,
                                          text=['0.01 R⊙', '0.1 R⊙', '1 R⊙', '10 R⊙', '100 R⊙', '1000 R⊙']))
    return LabelSet(x='x', y='y', text='text', level='glyph', x_offset=15, y_offset=-15, render_mode='canvas',
                    text_color='white', text_font_size='10px', text_align='right',
                    visible=False, source=label_source)

close_lum_labels  = lum_label_factory(df_close)
bright_lum_labels = lum_label_factory(df_bright)
both_lum_labels   = lum_label_factory(df_both)
either_lum_labels = lum_label_factory(df_either)
named_lum_labels  = lum_label_factory(df_named)

# placing isoradial luminosity labels at 'annotation' level ensures stars won't cover them up
close_lum_labels.level  = 'annotation' # 'image', 'underlay', 'glyph', 'guide', 'annotation' or 'overlay'
bright_lum_labels.level = 'annotation'
both_lum_labels.level   = 'annotation'
either_lum_labels.level = 'annotation'
named_lum_labels.level  = 'annotation'

plot.add_layout(close_lum_labels)
plot.add_layout(bright_lum_labels)
plot.add_layout(both_lum_labels)
plot.add_layout(either_lum_labels)
plot.add_layout(named_lum_labels)

In [58]:
text_color='lightcyan'

def labelset_factory(source, visible):
    return (LabelSet(x='B-V', y='M_V', text='Name', x_offset=2, y_offset=0,
                  text_font_size='11pt', text_color=text_color, text_align='center',
                  source=source, visible=visible),
            LabelSet(x='B-V', y='M_V', text='Name', x_offset=3, y_offset=-1,
                  text_font_size='11pt', text_color='black', text_align='center',
                  source=source, visible=visible))

close_labels, close_labels_shadow   = labelset_factory(source_close_labels,  True)
bright_labels, bright_labels_shadow = labelset_factory(source_bright_labels, False)
both_labels, both_labels_shadow     = labelset_factory(source_both_labels,   False)
either_labels, either_labels_shadow = labelset_factory(source_either_labels, False)
named_labels, named_labels_shadow   = labelset_factory(source_named_labels,  False)

plot.add_layout(close_labels_shadow)
plot.add_layout(close_labels)
plot.add_layout(bright_labels_shadow)
plot.add_layout(bright_labels)
plot.add_layout(both_labels_shadow)
plot.add_layout(both_labels)
plot.add_layout(either_labels_shadow)
plot.add_layout(either_labels)
plot.add_layout(named_labels_shadow)
plot.add_layout(named_labels)

In [59]:
tooltips='''
        <HTML>
        <HEAD>
        <style>
        .bk-tooltip {
            background-color: black !important;
            }
        </style>
        </HEAD>
        <BODY>
        <div>
            <div>
                <span style="color:blue; font-size: 12px; "><b>@Name</b></span><br>
                <span style="color:blue; font-size: 10px; ">Abs. mag.: @M_V{0.0}</span><br>
                <span style="color:blue; font-size: 10px; ">Radius: @{R_sun}{0.0} R⊙</span><br>
                <span style="color:blue; font-size: 10px; ">Luminosity: @{L_sun}{0.0} L⊙</span><br>
                <span style="color:blue; font-size: 10px; ">Distance: @{LY}{0.0} LY</span><br>
                <span style="color:blue; font-size: 10px; ">Spectral type: @SpType</span><br>
                <span style="color:blue; font-size: 10px; ">Color index: @{B-V}</span><br>
                <span style="color:blue; font-size: 10px; ">Eff. temp.: @{T_K}{0.0} K</span><br>
            </div>
        </div>
        </BODY>
        </HTML>'''

plot.add_tools(HoverTool(tooltips=tooltips,
                         show_arrow=False,
                         point_policy='snap_to_data',#'follow_mouse',
                         names=['stars'],
                         toggleable=True
                        ))

In [61]:
# making human-readable ticks for the log-scaled luminosity colorbar
logformatter = FuncTickFormatter(code="""
if (tick >= 0.1 && tick <=10) {
  return tick;
} else {
return 10 + (Math.log10(tick).toString()
             .split('')
             .map(function (d) { return d === '-' ? '⁻' : '⁰¹²³⁴⁵⁶⁷⁸⁹'[+d]; })
             .join(''));
}
""")

colorbar_color       = ColorBar(color_mapper=bv_color_mapper, location=(0,0), label_standoff=8,
                                title = 'B-V', padding=10, width = 20, visible=True)
colorbar_temperature = ColorBar(color_mapper=color_mapper_temperature, location=(0,0), label_standoff=8,
                                title = 'T in K', padding=5, width = 20,
                                ticker=LogTicker(desired_num_ticks=10, max_interval=10000, min_interval=10),
                                visible=False)
colorbar_luminosity  = ColorBar(color_mapper=color_mapper_luminosity, location=(0,0), label_standoff=8,
                                title = 'L⊙', padding=10, width = 20, formatter=logformatter,
                                ticker=LogTicker(desired_num_ticks=10, max_interval=100, min_interval=10),
                                visible=False)

plot.add_layout(colorbar_color,       'right')
plot.add_layout(colorbar_temperature, 'right')
plot.add_layout(colorbar_luminosity,  'right')

In [62]:
# html Div widgets
div_title = Div(text='''<b>Astro Lab Using HR Diagrams, <i>v. 1.0</i></b>''', width=400, height=20)

div_spacer = Div(text='', height=50)

div_hints = Div(text='''Things you can try: you can zoom in/out and pan on both plots with the mouse or gestures
on a mobile device. Scroll with the mouse wheel while hovering over an axis to zoom vert. or horiz. only (star
plot only). If the stars' tooltip pop-ups are distracting, they can be turned off with the <b>Hover</b> tool in
the toolbar above the plot. There are buttons to <b>Reset</b> and <b>Save</b> the plot there as well. Below the
star plot are options for which sets of stars to plot and which color scheme to use.''',
          width=400, height=140)

div_links = Div(text='''In appreciation of UNL's <a href="https://astro.unl.edu" rel="noopener noreferrer"
target="_blank">Astronomy page</a> and great but soon-to-expire Flash-based
<a href="https://astro.unl.edu/naap/hr/animations/hr.html" rel="noopener noreferrer"target="_blank"
>HR Diagram explorer</a>.<br>Check out my <a href="https://skyfield-orbits.herokuapp.com/skyfield-orbits" 
rel="noopener noreferrer"target="_blank">solar system animation</a>, now with retrograde! See my
<a href="https://github.com/swshadle/physics"rel="noopener noreferrer"target="_blank">GitHub</a>
for contact info and more projects.''', width=400, height=50)

In [63]:
p2 = figure(plot_width=420, plot_height=400, active_drag='pan', background_fill_color='black',
            x_range=(-2.35, 2.1), y_range=(-2.1, 2.1), tools='pan,zoom_in,zoom_out,reset')

p2.add_tools(wheel_zoom)
p2.toolbar.active_scroll = wheel_zoom
p2.grid.grid_line_color = None
p2.toolbar_location='above'
p2.axis.visible=False
p2.toolbar.autohide = True
p2.toolbar.logo=None

In [65]:
# create a data source to enable refreshing of fill & text color
bv_source = ColumnDataSource(data=dict(color=[BV_sun]))
bv_source_sun = ColumnDataSource(data=dict(color=[BV_sun]))
bv_color_mapper = LinearColorMapper(palette=blu_or, low=start_bv, high=end_bv)
star_rad = 1
render_circle = p2.circle(-0.25-star_rad, 0, radius=star_rad, alpha=1, line_color='black',
                          fill_color={'field': 'color', 'transform': bv_color_mapper}, source=bv_source)
p2.circle(1, 0, radius=1, alpha=1,  line_color='black',
          fill_color={'field': 'color', 'transform': bv_color_mapper}, source=bv_source_sun)

sun_label  = Label(x=p2.plot_width*0.90, y=10, x_units='screen', y_units='screen', text_align='right',
                   background_fill_color='black', text_color='white', background_fill_alpha=.5, text=' sun ')
star_label = Label(x=p2.plot_width*0.03, y=10, x_units='screen', y_units='screen', text_align='left',
                   background_fill_color='black', text_color='white', background_fill_alpha=.5, text=' star ')

p2.add_layout(sun_label)
p2.add_layout(star_label)

In [66]:
# under the hood, the temperature slider has color index (B-V) values
bv_slider = Slider(title='Temperature', start=start_bv, end=end_bv, step=0.001, width=400,
                   value=np.around(BV_sun, 3), # round starting value to 3 decimal places
                   format=FuncTickFormatter(code="""return (5601/Math.pow(tick+0.4,2/3)).toFixed(0) + ' K'"""))

lum_slider = Slider(title='Luminosity', start=start_lum, end=end_lum, step=.1, width=400,
                    value=0, format=FuncTickFormatter(code="""return Math.pow(10,tick).toPrecision(4) + ' L⊙'"""))

We want to show slider values with a marker on the HR plot. The sliders' temperature and luminosity can be converted to the HR plot's color index (x-axis) and abs. magnitude (y-axis) using equation (3) for `M_V` and solving equation (5) for `B-V`.<br>
$$M_V = M_V☉ - 2.5 \cdot log\frac{L}{L_{sun}}\tag{3}$$<br>
$$T_K = \frac{5601 K}{(B-V+0.4)^{2/3}}\tag{5}$$<br>
$$(B-V+0.4)^{2/3} = \frac{5601 K}{T_K}$$<br>
$$B-V+0.4 = \left(\frac{5601 K}{T_K}\right)^{3/2}$$<br>
$$B-V = \left( \frac{5601 K}{T_K} \right) ^{3/2} - 0.4\tag{6}$$

In [67]:
def t_from_color(c):
    return 5601/np.power(c+0.4,2/3)

temp = t_from_color(bv_slider.value)

from latex_label import LatexLabel
latex = LatexLabel(
    text = f'\mathrm{{radius}} = \
    {{\sqrt{{L/L_\odot}} \over (T/T_\odot)^2}} = \
    {{\sqrt{{1}} \over ({{{T_sun}}}/{T_sun})^2}} = \
    {1} R_\odot',
    x=0,
    y=-10,
    x_units='screen',
    y_units='screen',
    render_mode='css', # Note: must set ``render_mode="css"``
    text_font_size='14px',
    text_color='black',
    background_fill_alpha=0,
)
p2.add_layout(latex)

# debug_label = Label(x=10, y=p2.plot_height/2, x_units='screen', y_units='screen', text_align='left',#'data',
#                     text_color='white', background_fill_color='black', text_font_size='16px',
#                     background_fill_alpha=0.8, text='Debug')
# p2.add_layout(debug_label)

In [68]:
slider_callback_js = CustomJS(args=dict(bv_source=bv_source, bv_slider=bv_slider,
                                        lum_slider=lum_slider, latex=latex,
                                        render_circle=render_circle,
#                                         label=debug_label, 
                                        render_marker=render_marker),
                              code="""
    var lum = Math.pow(10,lum_slider.value).toPrecision(4); // + ' L⊙';
    var temp = ~~5601/Math.pow(bv_slider.value+0.4,2/3);   // ).toFixed(0) + ' K' // or Math.round() if double bit-inversion (~~) doesn't work
    var rad = Math.sqrt(lum)/(temp*temp/33408400); // 5780**2 = 33408400

    const color = bv_source.data['color'];
    color[0] = bv_slider.value;
    bv_source.change.emit();
    // label.text = 'BV value: ' + bv_slider.value.toPrecision(3) + ' lum value: ' + lum_slider.value.toPrecision(3);
    // latex.text = 'radius = √(' + lum + ')/(' + temp.toFixed(0) + '/5780)² = ' + rad.toPrecision(5) + ' R⊙';
    // latex.text = f'\mathrm{{radius}} = {{\sqrt{{L/L_\odot}} \over (T/T_\odot)^2}} = {{\sqrt{{{lum_str}}} \over ({{{int(temp)}}}/{T_sun})^2}} = {rad_str} R_\odot'
    render_circle.glyph.radius = rad;
    render_circle.glyph.x = -0.25 - rad;
    render_marker.glyph.x = parseFloat(bv_slider.value);
    render_marker.glyph.y = parseFloat(4.83 - 2.5*lum_slider.value);
""")

def solar_radii_str(lu, te):
    return np.format_float_positional(np.sqrt(lu)/(te/T_sun)**2, precision=3, unique=False, trim='-',
                                      fractional=False, sign=False, pad_left=None, pad_right=None)

def slider_callback(attr, old, new): # standard attr, old, new parameters
    lum_str = np.format_float_positional(10.0**lum_slider.value, precision=3, unique=False,
                                         trim='-', fractional=False, sign=False, pad_left=None,
                                         pad_right=None)
    temp = t_from_color(bv_slider.value)
    rad_str = solar_radii_str(float(lum_str), temp)
    latex.text = f'\mathrm{{radius}} = {{\sqrt{{L/L_\odot}} \over (T/T_\odot)^2}} = \
        {{\sqrt{{{lum_str}}} \over ({{{int(temp)}}}/{T_sun})^2}} = {rad_str}\: R_\odot'

bv_slider.js_on_change('value', slider_callback_js)
bv_slider.on_change('value', slider_callback)
lum_slider.js_on_change('value', slider_callback_js)
lum_slider.on_change('value', slider_callback)

button = Button(label='Reset to solar temperature and luminosity', button_type="success")

def button_callback():
    latex.text = f'\mathrm{{radius}} = \
    {{\sqrt{{L/L_\odot}} \over (T/T_\odot)^2}} = \
    {{\sqrt{{1}} \over ({{{T_sun}}}/{T_sun})^2}} = 1\: R_\odot'
    
button.on_click(button_callback)

button.js_on_click(CustomJS(args=dict(bv_source=bv_source, bv_slider=bv_slider, latex=latex,
                                      lum_slider=lum_slider, BV_sun=BV_sun,
                                      render_circle=render_circle,
#                                       label=debug_label,
                                      render_marker=render_marker,
                                      M_sun=M_sun),
                            code="""
    // label.text = 'Attempting to set bv_slider to ' + BV_sun.toFixed(3);
    render_circle.glyph.radius = 1
    render_circle.glyph.x = -0.25 - 1
    bv_slider.value = parseFloat(BV_sun.toFixed(3));
    lum_slider.value = 0.0;
    // latex.text = 'radius = √(1)/(5780/5780)² = 1 R⊙';
    // latex.text = f'\mathrm{{radius}} = {{\sqrt{{L/L_\odot}} \over (T/T_\odot)^2}} = {{\sqrt{{{lum_str}}} \over ({{{int(temp)}}}/{T_sun})^2}} = {rad_str} R_\odot'
    // label.text = 'BV value: ' + bv_slider.value.toPrecision(3) + ' lum value: ' + lum_slider.value.toPrecision(3);

    const color = bv_source.data['color'];
    color[0] = bv_slider.value;
    bv_source.change.emit();
    render_marker.glyph.x = BV_sun; // bv_slider.value;
    render_marker.glyph.y = M_sun; // lum;
"""))

In [69]:
curdoc().clear()
curdoc().title = f'Hertzsprung-Russell'
layout = row(
    column(plot, Paragraph(text='Show stars', width=70, height=10), row(radiogroup, column(select_mapper, checkboxgroup))),
    column(div_title, p2, div_spacer, bv_slider, lum_slider, button, div_hints, div_links))
curdoc().add_root(layout)

In [73]:
# df[df['Name'].str.contains('Barn')==True] # looking for Barnard's Star

In [74]:
# df[df['Name'].str.contains('Sun')==True] # looking for the sun